In [7]:
import os.path

import cv2
import numpy as np
import pandas as pd
from scipy.stats import norm
import random
from sensor_projection_utils import equirect_point_to_pixels

In [9]:
def get_fov_saliency_rate_and_target_point(frame, center, fov,previous_intr_coeff,change_significance_threshold=0.02,intr_threshold=0.95):
    
    gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    intensity_list = []
    rect_width, rect_height = fov
    max_intensity = 0

    # Define the rectangle boundaries
    left = max(center[0] - rect_width // 2, 0)
    right = min(center[0] + rect_width // 2, frame.shape[1])
    
    #detect if there's a wraping in the equirectangular frame FoV (accounting for sphere movement)
    left_rect  = False
    right_rect = False
    if left == 0:
        right_rect = True
        print("right wrap case")
    if right == frame.shape[1]:
        left_rect = True
        print("left wrap case")
   
        
        
    
    top = max(center[1] - rect_height // 2, 0)
    bottom = min(center[1] + rect_height // 2, frame.shape[0])

    for x in range(left, right):
        for y in range(top, bottom):
            intensity = gray_frame[y, x]
            intensity_list.append(((x, y,intensity), intensity))
 
    if left_rect:
        rect_wid =  rect_width // 2
        for x in range(1, rect_wid):
            for y in range(top, bottom):
                intensity = gray_frame[y, x]
                intensity_list.append(((x, y,intensity), intensity))
    
    if right_rect:
        rect_wid =  rect_width // 2
        for x in range(frame.shape[1] - rect_wid, frame.shape[1]):
            for y in range(top, bottom):
                intensity = gray_frame[y, x]
                intensity_list.append(((x, y,intensity), intensity))
     

        
    random.shuffle(intensity_list)     
    
    low_intensities_count = sum(1 for intensity in intensity_list if intensity[1] < 30)
 
    low_percentage = (low_intensities_count / len(intensity_list))
    
    #check if there's no movement
    if (np.abs(low_percentage - previous_intr_coeff)  <= change_significance_threshold and low_percentage <intr_threshold):
        return None,low_percentage
        
    
    # Sort by intensity and pick the top 3
   
    top_3 = sorted(intensity_list, key=lambda x: x[1], reverse=True)[:3]

    if len(intensity_list) == 0 :
        return center
    #return target point
    return random.choice(top_3)[0],low_percentage

In [10]:
def simulation(input_video_env_path,output_trajectory_csv_path, frame_range, X_Mov_law, Y_Mov_law, fov_dimensions=(720,720),change_significance_threshold=0.02,intr_threshold=0.95 , adjustment_threshold=0.98):
    
    

    df = pd.DataFrame(columns=["Frame.No",'X','Y','X_pixel','Y_pixel','X_mov','Y_mov'])

    cap = cv2.VideoCapture(input_video_env_path)
    width, height = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH)), int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    
    frame_count = 0
    
    #randomly initialize initial agent movement
    X, Y = norm(loc=0.63, scale=0.29).rvs(),norm(loc=0.52, scale=0.13).rvs()

    dx ,dy = 0,0
    
    intrest_rate = 1
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret or frame_count > frame_range[1]:
            break
        if frame_count >= frame_range[0]:
            
            # the norm(loc=0.0, scale=0.001) is the noise
            X += dx + norm(loc=0.0, scale=0.001).rvs()
            Y += dy + norm(loc=0.0, scale=0.001).rvs()
            
            #Frame wrapping
            if X>1.25:
                X = (X - 1.25) + 0.25 
            if X<0.25:
                X = 1.25 - abs(0.25 - X)  
            
            X_pixel, Y_pixel = equirect_point_to_pixels([X,Y],[width,height]) 
            X_pixel, Y_pixel = int(X_pixel) , int(Y_pixel)
           
            
            
            df.loc[len(df)] = [frame_count, X, Y, X_pixel, Y_pixel, dx, dy]                
      
            print(f'frame is {frame_count}')
             
                  
            if frame_count % 10 == 0:
                
                X_mov , Y_mov = 0,0
                target_point,intrest_rate = get_fov_saliency_rate_and_target_point(frame, (X_pixel, Y_pixel), fov_dimensions , intrest_rate,change_significance_threshold=0.02,intr_threshold=0.95)
                
                if target_point is not None:
                    X_mov = np.abs(norm(loc=X_Mov_law[0], scale=X_Mov_law[1]).rvs())
                    Y_mov = np.abs(norm(loc=Y_Mov_law[0], scale=Y_Mov_law[1]).rvs())    
                   
                    if intrest_rate <adjustment_threshold:
                        #Movement direction         
                        if target_point[0] < X_pixel:
                            X_mov = X_mov * -1
                        if target_point[1] < Y_pixel:
                            Y_mov = Y_mov * -1
                    
                    #adjsut agent to the center of the environment
                    elif intrest_rate >=adjustment_threshold:
                         center = int(height / 2) 
                         
                                                       
                         if Y_pixel<center:
                             Y_mov = Y_mov*2
                         else :    
                             Y_mov = Y_mov*-2
                            
                dx = X_mov /10 
                dy = Y_mov / 10 

        frame_count += 1
        
    df.to_csv(output_trajectory_csv_path,index=False)        
    cap.release()

In [ ]:
for i in range(1,49):
    print("-----------------------------")
    print(f"Synthetic Agent {i}")
    simulation("./002-sal-eml.mp4",f"./sal2-eml/002-syn-u{i}.csv", [1,1800],[0.013327,0.023678], [0.0073,0.01352], [720,720])

In [18]:
#draw in a frame a rectangle around a fixation point representing the simplified FoV of a user
def draw_fov(frame, center, dimensions,border_color=(0, 255, 0)):
    rect_width, rect_height = dimensions
    top_left = (center[0] - rect_width // 2, center[1] - rect_height // 2)
    bottom_right = (center[0] + rect_width // 2, center[1] + rect_height // 2)
    cv2.rectangle(frame, top_left, bottom_right, border_color, 2)

def simulation_visualization(video_path, trajectory_path, output_video_path, fixation_color=(0, 0, 255), fov_dimensions=(720, 720),fov_border_color=(0, 255, 0)):
    """
    Reads a DataFrame with frame information, processes a video,
    draws fixations and FoV on specified frames,representing user head movements and saves a new video.
    """

    df = pd.read_csv(trajectory_path)

    cap = cv2.VideoCapture(video_path)
    frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps = cap.get(cv2.CAP_PROP_FPS)
    out = cv2.VideoWriter(output_video_path, cv2.VideoWriter_fourcc(*'mp4v'), fps, (frame_width, frame_height))

    for index, row in df.iterrows():
        frame_no = row['Frame.No'] - 1  # Adjust for zero-based indexing
        print(frame_no)
        
        cap.set(cv2.CAP_PROP_POS_FRAMES, frame_no)
        ret, frame = cap.read()
    
        if ret:
            x_pixel = int(row['X_pixel'])
            y_pixel = int(row['Y_pixel'])
    
            cv2.circle(frame, (x_pixel, y_pixel), 15, fixation_color, -1)  # Draw point
            draw_fov(frame, [x_pixel,y_pixel], fov_dimensions)
    
            out.write(frame)

    cap.release()
    out.release()


In [ ]:
simulation_visualization("./002-sal-eml.mp4", "./sal2-eml/002-syn-u1.csv", "./trajectory-video2-user1.mp4", fixation_color=(0, 0, 255), fov_dimensions=(720, 720),fov_border_color=(0, 255, 0))